# <center>Classification des Tweets</center>


<center><img alt="badbuzz" src="images/pn.png"/></center><br>

Nous allons prédire le sentiment de Tweets (« positif » ou « négatif ») en utilisant plusieurs types de modèles. 
Nous prendrons un échantillon de 20 000 Tweets sur les 1 600 000 disponibles afin de gagner en efficacité sur les temps de traitement avec des performances acceptables pour certains modèles. Il y aura donc 10 000 Tweets positifs et 10 000 Tweets négatifs sélectionnés afin qu’une classe ne soit pas considérée comme « plus importante » lors de la création des modèles.<br> Nous sommes dans une démarche de classification supervisée, par conséquent on séparera les 20 000 tweets 3 parties :<br>
•	Jeu d’entrainement : 12800 Tweets : 64%<br> 
•	Jeu de validation : 3200 Tweets : 16%<br>
•	Jeu de test : 4000 Tweets : 20% <br>
Chaque jeu contiendra autant de Tweet positifs que négatifs.<br>

## 1.	Modèles sur-mesure simple<br>
Nous allons aborder 3 types de modèles : <br>
•	eXtreme Gradient Boosting<br>
•	Multinomial Naïve Bayse<br>
•	Logistic Regression<br><br>
Ces modèles prennent en compte des valeurs numériques et non du texte « brut ». Par conséquent, chacun des Tweets sera soumis à un plongement de mot (embedding) **« Bag-of-Words »** et **« TF-IDF »** pour observer l’influence de ces transformations sur les modèles de ces méthodes. Aussi, chaque modèle est soumis à la recherche des meilleurs hyperparamètres via GRID-SEARCH CV complétée de l’option StratifiedKFold.<br><br>
### **1.1	eXtreme Gradient Boosting**<br>
Cet algorithme fonctionne de manière séquentielle. Il va créer un premier modèle qu’il va évaluer. A partir de cette première évaluation, l’algorithme va pondérer les individus en fonction de la performance de leur prédiction et va ensuite recréer un nouveau modèle. Pour notre échantillon, ce modèle génère un AUC=0.7 sur les 2 embeddings cités précédemment.<br><br>
### **1.2	Multinomial Naïve Bayse**<br>
Cet algorithme est idéal pour la classification de données textuelles (via une discrétisation de l’effectif des mots). Selon Bayse, le calcul de la probabilité conditionnelle consiste à calculer la probabilité de l’évènement étant donné l’occurrence d’un autre évènement. Cependant, l’algorithme vise à simplifier le calcul en considérant que chaque variable est indépendante des autres et par ce fait, chaque probabilité de chaque variable est calculée indépendamment.   L’aspect multinomial permet d’interpréter la fréquence des mots. Pour notre échantillon, ce modèle génère un AUC=0.73 (embedding TF-IDF) et AUC = 0.72 (embedding Bag-of-Word).<br><br>
### **1.3	Logistic Regression**<br>
Ce modèle estime la probabilité qu’un événement ait lieu sur la base d’un ensemble de variables explicatives c’est-à-dire un ensemble d’évènements susceptibles d’influencer cet événement. Le résultat du classement étant une probabilité, si la valeur est supérieure à 0.5, le Tweet est considéré comme positif et si la valeur est inférieure à 0.5, il est considéré comme négatif. Pour notre échantillon, ce modèle génère un AUC=0.72 sur les 2 embeddings cités précédemment.<br><br>
### **Conclusion**<br>
On remarque que les résultats générés par ces 3 classifieurs sont presque identiques avec le meilleur score proposé par le Multinomial Naïve Bayse et un embedding en TF-IDF : AUC=0.73

<center><img alt="AUC" src="images/modele/mnb/tf/auc_mnb_tf.png"/></center><br>
<center><img alt="AUC" src="images/modele/mnb/tf/class_report_mnb_tf.png"/></center><br>

Voici un résumé des résultats obtenu pour chacun des modèles :<br>

<table>
      <tr>
        <th><center>Modèle</center></th>
        <th><center>Embedding</center></th>
        <th><center>AUC</center></th>
      </tr>
      <tr>
        <td><center>XGB</center></td>
        <td><center>BoW</center></td>
        <td><center>0.70</center></td>
      </tr>
      <tr>
        <td><center>XGB</center></td>
        <td><center>TF-IDF</center></td>
        <td><center>0.70</center></td>
      </tr>
      <tr>
        <td><center>NMF</center></td>
        <td><center>BoW</center></td>
        <td><center>0.72</center></td>
      </tr>
        <tr>
        <td><center>NMF</center></td>
        <td><center>TF-IDF</center></td>
        <td><center>0.73</center></td>
        </tr>
       <tr>
        <td><center>LR</center></td>
        <td><center>BoW</center></td>
        <td><center>0.72</center></td>
       </tr>
          <tr>
        <td><center>LR</center></td>
        <td><center>TF-IDF</center></td>
        <td><center>0.72</center></td>
      </tr>
</table>

## 2.	Modèles sur-mesure avancés
Nous allons utiliser des modèles basés sur des réseaux de neurones récurrents (RNN : recurrent neural network) qui sont adaptés à des entrées de tailles variables (ici des Tweets de longueurs différents). En effet ces réseaux contiennent des neurones pour lesquels leur sortie est connectée à l’entrée du neurone afin de considérer les évènements passés. Cependant, les RNNs se heurtent au problème de disparition du gradient pour apprendre à mémoriser des évènements passés. 
Par conséquent pour parlier à ce problème il a été créé le réseau LSTM (Long Short-Term Memory) qui possède une cellule mémoire interne pilotée par 3 portes :<br> 
•	La porte d'entrée décide si l'entrée doit modifier le contenu de la cellule<br> 
•	La porte d'oubli décide s'il faut remettre à 0 le contenu de la cellule<br> 
•	La porte de sortie décide si le contenu de la cellule doit influer sur la sortie du neurone<br><br> 
Dans le cas de l’utilisation du LSTM, les sorties de tous les neurones sont réinjectées en entrées de tous les neurones ; de plus les connexions nécessaires au pilotage des cellules mémoire engendrent un réseau très « lourd » comparés aux couches RNN simples. Afin de réduire le nombre de paramètre du réseau, nous allons aussi utiliser une variante du LSTM appelée GRU (Gated Recurrent Unit) qui supprime la porte d’oubli. <br> 
Dans le cadre de notre étude nous allons donc utiliser et analyser les performances des 3 modèles suivants :<br> 
•	**Simple RNN**<br> 
•	**LSTM**<br> 
•	**GRU**<br><br> 
Comme pour les modèles sur-mesure simples vus précédemment, nous devons transformer les Tweets en valeurs numériques. Dans un premier temps, nous allons juste utiliser un Tokenizer sur les 3 modèles puis nous allons utiliser indépendamment les 3 types « d'embedding » suivants :<br> 
•	**Word2Vec**<br>
•	**GloVe**<br>
•	**FastText**<br>
•	**USE**<br> <br> 
Ces embedding seront ajoutés aux 3 modèles sous forme de couches pré-entrainées et permettront de prendre en compte différents aspects d’un texte comme la position des mots dans le texte (syntaxe), la signification des mots (sémantique), la prise en compte du sens du mot pouvant en avoir plusieurs en fonction du contexte (polysémie). L’embedding USE se place plutôt sur un niveau de la signification des phrases plutôt que des mots individuellement. Par conséquent, certains embeddings sont plus ou moins consommateur de ressources.<br>
Chaque embedding a des méthodes de fonctionnement différents : <br>  
•	**Word2Vec** : crée par Google, sont des réseaux de neurones artificiels à deux couches entraînées pour reconstruire le contexte linguistique des mots. Le modèle utilise 2 architectures pour le calcul des vecteurs : CBOW (prédiction du mot en fonction du contexte) et Skip-gram (prédiction du contexte en fonction du mot)<br> 

•	**GloVe** : Global Vectors for word representation : crée par Stanford University, cette méthode de plongement utilise la matrice de cooccurrences qui indique la fréquence à laquelle une paire de mots particulière apparaît ensemble<br> 

•	**FastText** : c’est une extension à l’embedding Word2vec ; Word2Vec fourni un plongement de mots alors que FastText fourni un plongement sur n-grams apportant ainsi des informations sur la structure du langage<br> 

•	**Universal Sentence Encoder (USE)** : encodage sur la phrase complète au lieu d’encoder mot par mot. On peut ainsi étudier la similarité de sens au niveau de phrase et permettant ainsi une meilleure performance sur les taches de classification en utilisant moins de données d’entrainement supervisés.<br> 

### Conclusion 

Dans notre contexte, on remarque que le modèle GRU avec l'embedding GloVe apporte la meilleure performance (AUC = 0.76)<br>
<center><img alt="AUC" src="images/modele/rnn/gru/glove/rnn_gru_glove_auc.png"/></center><br>
<center><img alt="AUC" src="images/modele/rnn/gru/glove/rnn_gru_glove_class_report.png"/></center><br>


Voici les performances enregistrées pour chacun des modèles avec l'embedding associé :<br>



<table>
      <tr>
        <th><center>Modèle</center></th>
        <th><center>Embedding</center></th>
        <th><center>AUC</center></th>
      </tr>
      <tr>
        <td><center>Simple RNN</center></td>
        <td><center>Aucun</center></td>
        <td><center>0.62</center></td>
      </tr>
      <tr>
        <td><center>LSTM</center></td>
        <td><center>Aucun</center></td>
        <td><center>0.62</center></td>
      </tr>
      <tr>
        <td><center>GRU</center></td>
        <td><center>Aucun</center></td>
        <td><center>0.62</center></td>
      </tr>
      <tr>
        <td><center>Simple RNN</center></td>
        <td><center>GloVe</center></td>
        <td><center>0.71</center></td>
      </tr>
      <tr>
        <td><center>LSTM</center></td>
        <td><center>GloVe</center></td>
        <td><center>0.75</center></td>
      </tr>
      <tr>
        <td><center>GRU</center></td>
        <td><center>GloVe</center></td>
        <td><center>0.76</center></td>
      </tr>
          <tr>
        <td><center>Simple RNN</center></td>
        <td><center>Word2Vec</center></td>
        <td><center>0.51</center></td>
      </tr>
      <tr>
        <td><center>LSTM</center></td>
        <td><center>Word2Vec</center></td>
        <td><center>0.59</center></td>
      </tr>
      <tr>
        <td><center>GRU</center></td>
        <td><center>Word2Vec</center></td>
        <td><center>0.58</center></td>
      </tr>
          <tr>
        <td><center>Simple RNN</center></td>
        <td><center>FastText</center></td>
        <td><center>0.51</center></td>
      </tr>
      <tr>
        <td><center>LSTM</center></td>
        <td><center>FastText</center></td>
        <td><center>0.51</center></td>
      </tr>
      <tr>
        <td><center>GRU</center></td>
        <td><center>FastText</center></td>
        <td><center>0.53</center></td>
      </tr>
          <tr>
        <td><center>Simple RNN</center></td>
        <td><center>USE</center></td>
        <td><center>0.58</center></td>
      </tr>
      <tr>
        <td><center>LSTM</center></td>
        <td><center>USE</center></td>
        <td><center>0.65</center></td>
      </tr>
      <tr>
        <td><center>GRU</center></td>
        <td><center>USE</center></td>
        <td><center>0.64</center></td>
      </tr>
</table>

## 3.	Modèle avancé BERT
Bidirectional Encoder Representations from Transformers (BERT), développé par Google AI, permet de représenter un mot de façon contextuelle, c’est-à-dire que chaque mot est représenté en fonction de son sens dans le contexte rencontré. Par exemple le « rouge » aura une représentation différente dans « J’achète du rouge pour les invités » et « La voiture sera peinte en rouge ». De plus BERT utilise une technique appelée **Masked LM (MLM)**, qui consiste à masquer aléatoirement des mots dans une phrase et essai de les prédire en utilisant le contexte de la phrase à gauche et à droite du mot masqué (aspect bidirectionnel) et ce de manière simultanée. BERT applique un mécanisme d'attention pour comprendre les relations entre les mots de la phrase ; pour exemple, si l'on prend la phrase "tu as une souris sur ton ordinateur", BERT va prêter attention au mot "ordinateur" pour déterminer le sens du mot "souris".<br>
Dans notre contexte, le modèle de BERT nous permet d'obtenir un AUC de 0.88 :<br>
<center><img alt="AUC" src="images/modele/bert/m1_TFBertForSequenceClassification/88/auc_bert_m1.png"/></center><br>
<center><img alt="AUC" src="images/modele/bert/m1_TFBertForSequenceClassification/88/class_report_bert_m1.png"/></center><br>


# <center>Démarche MLOps<center>

<center><img alt="MLOPS" src="images/mlops.png"/></center><br>


Après avoir récupéré les données puis procédé au nettoyage et à l’analyse (EDA : Exploratory Data Analysis), la création de plusieurs types de modèles avec l’utilisation de plusieurs types embeddings a été réalisé pour classer les Tweets (positif ou négatif). Chaque modèle est enregistré sur MLFLOW ainsi que leurs performances associées.<br><br>
Ensuite un pipeline de déploiement continu a été réalisé. En effet, le meilleur modèle sélectionné a été déposé sur GitHub et via **GitHub actions** le modèle et l'API FLASK associés vont être automatiquement déposés sur un serveur en ligne (Heroku). Grâce au développement de l’API via **FLASK**, on peut interroger notre modèle (déposé sur le serveur en ligne) en lui envoyant un Tweet ; on obtiendra ainsi une réponse « positif » ou « négatif » afin de connaitre la classification du Tweet envoyé. A chaque nouveau jeu de données reçu, un nouveau modèle est créé, des tests unitaires sont lancés pour vérifier si tout s’est bien passé ; si le dernier modèle crée est plus performant que le précédent, le processus cité dans le dernier paragraphe est renouvelé. <br>
Voici plus précisément le pipeline déployé pour ce projet : <br>
<center><img alt="pipeline" src="images/pipeline.png"/></center><br>

# FIN